In [76]:
#import neccessary modules
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

In [77]:
# helper functions
from timeseries_helpers import datasetstorer
from timeseries_helpers import dataset_importer
from timeseries_helpers import database_importer
from timeseries_helpers import standardizer
from data_preprocessing import extract_accelerometer_data, csv_to_dataset_list

In [78]:
ds = csv_to_dataset_list('/Users/raouldoublan/Documents/GitHub/Boxschlag-Klassifikation/data/Max_Gerade.csv')

    



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18485 entries, 0 to 18484
Data columns (total 4 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   accelerometerTimestamp_sinceReboot(s)  18485 non-null  float64
 1   accelerometerAccelerationX(G)          18485 non-null  float64
 2   accelerometerAccelerationY(G)          18485 non-null  float64
 3   accelerometerAccelerationZ(G)          18485 non-null  float64
dtypes: float64(4)
memory usage: 577.8 KB
Data imported without any errors!
 ##### Information: ####
 Imported datasets: 1


In [79]:
len(ds)

1

In [80]:
# converts the given list of dataframes to one dataframe
df_list = []
for idx,e in enumerate(ds):
    df = e.assign(punch_id=idx)
    df['label'] = 0
    df_list.append(df)
df_res = pd.concat(df_list)
df_new = df_res.reset_index(drop=True)
df_new = df_new.rename(index=str, columns={"accelerometerAccelerationX(G)": "a_x", "accelerometerAccelerationY(G)": "a_y", "accelerometerAccelerationZ(G)":"a_z"})

In [81]:
len(df_new)

18485

In [82]:
df_new.head()

,accelerometerTimestamp_sinceReboot(s),a_x,a_y,a_z,punch_id,label
0,15981.514496,-0.059662,-0.098602,-0.960510,0,0
1,15981.524465,-0.021240,-0.115128,-0.966843,0,0
2,15981.534434,-0.059235,-0.121017,-0.965866,0,0
3,15981.544403,-0.051376,-0.108932,-0.978439,0,0
4,15981.554372,-0.038437,-0.100632,-0.986923,0,0


FileNotFoundError: [Errno 2] No such file or directory: '../../Datasets/complete_smartpunch_dataset_AsOneDataFrame_periodMS2000_sampleUS1000.csv'